# 前期处理

In [1]:
# coding: utf-8
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from category_encoders.count import CountEncoder
from tqdm import tqdm
from tqdm import tqdm_notebook
import gc
warnings.filterwarnings('ignore')

In [2]:
#请换成您的数据所在路径
DATA_PATH="F:/Desktop/data/"
train_label = pd.read_csv(DATA_PATH+'train_label.csv')
train_base = pd.read_csv(DATA_PATH+'train_base.csv')
test_a_base = pd.read_csv(DATA_PATH+'test_a_base.csv')
test_b_base=pd.read_csv(DATA_PATH+"testb_base.csv")
train_op = pd.read_csv(DATA_PATH+'train_op.csv')
train_trans = pd.read_csv(DATA_PATH+'train_trans.csv')
test_a_op = pd.read_csv(DATA_PATH+'test_a_op.csv')
test_b_op=pd.read_csv(DATA_PATH+"testb_op.csv")
test_a_trans = pd.read_csv(DATA_PATH+'test_a_trans.csv')
test_b_trans=pd.read_csv(DATA_PATH+"testb_trans.csv")

In [3]:
#定义生成时间戳函数
def transform_time(x):
    day = int(x.split(' ')[0])
    hour = int(x.split(' ')[2].split('.')[0].split(':')[0])
    minute = int(x.split(' ')[2].split('.')[0].split(':')[1])
    second = int(x.split(' ')[2].split('.')[0].split(':')[2])
    return 86400*day+3600*hour+60*minute+second

In [4]:
#复制数据
train_df = train_base.copy()
test_a_df = test_a_base.copy()
test_b_df = test_b_base.copy()
train_df = train_label.merge(train_df, on=['user'], how='left')
del train_base, test_a_base, test_b_base
#合并数据
op_df = pd.concat([train_op, test_a_op,test_b_op], axis=0, ignore_index=True)
trans_df = pd.concat([train_trans, test_a_trans,test_b_trans], axis=0, ignore_index=True)
data = pd.concat([train_df, test_a_df,test_b_df], axis=0, ignore_index=True)
del train_op, test_a_op, test_b_op, train_df, test_a_df, test_b_df, train_trans, test_a_trans,test_b_trans
# 时间维度的处理
op_df['day'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
trans_df['day'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
op_df['timestamp'] = op_df['tm_diff'].apply(lambda x: transform_time(x))
trans_df['timestamp'] = trans_df['tm_diff'].apply(lambda x: transform_time(x))
op_df['hour'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
trans_df['hour'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
trans_df['week'] = trans_df['day'].apply(lambda x: x % 7)
op_df['min'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[1]))
op_df['second'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[2]))
# 排序
trans_df = trans_df.sort_values(by=['user', 'timestamp'])
op_df = op_df.sort_values(by=['user', 'timestamp'])
trans_df.reset_index(inplace=True, drop=True)
op_df.reset_index(inplace=True, drop=True)

gc.collect()

0

In [5]:
#按小时对交易表进行时间分组
trans_df["time"]="time"
trans_df.loc[trans_df.hour<=6,"time"]="night"
trans_df.loc[(trans_df.hour>6)&(trans_df.hour<=12),"time"]="morning"
trans_df.loc[(trans_df.hour>12)&(trans_df.hour<=18),"time"]="afternoon"
trans_df.loc[trans_df.hour>18,"time"]="evening"
#按小时对op表进行时间分组
op_df["time"]="time"
op_df.loc[op_df.hour<=6,"time"]="night"
op_df.loc[(op_df.hour>6)&(op_df.hour<=12),"time"]="morning"
op_df.loc[(op_df.hour>12)&(op_df.hour<=18),"time"]="afternoon"
op_df.loc[op_df.hour>18,"time"]="evening"

In [6]:
#删除缺失过多的service3_level
data.drop(['service3_level'], axis=1, inplace=True)

# 特征工程

In [7]:
#算术特征提取
data['product7_fail_ratio'] = data['product7_fail_cnt'] / data['product7_cnt']
data['city_count'] = data.groupby(['city'])['user'].transform('count')
data['province_count'] = data.groupby(['province'])['user'].transform('count')

In [8]:
#对年龄和使用时长进行woe编码
def woe_iv_of_value(value):
    df=data[~data['label'].isnull()].copy()
    df['label_0'] = df['label'].map(lambda x: 1 if x == 0 else 0)
    df['label_1'] = df['label'].map(lambda x: 1 if x == 1 else 0)
    woe_iv_df_1 = df.groupby(value)['label_0'].agg({'sum'}).reset_index()
    woe_iv_df_1.columns=[value,"cnt_0_of_{}".format(value)]
    woe_iv_df_2 = df.groupby(value)['label_1'].agg({'sum'}).reset_index()
    woe_iv_df_2.columns=[value,"cnt_1_of_{}".format(value)]
    woe_iv_df = pd.merge(woe_iv_df_1, woe_iv_df_2, how = 'left', on = value)
    woe_iv_df['cnt_of_{}'.format(value)] = woe_iv_df["cnt_0_of_{}".format(value)] + woe_iv_df["cnt_1_of_{}".format(value)]
    woe_iv_df['ratio_0_of_{}'.format(value)] = woe_iv_df["cnt_0_of_{}".format(value)]/woe_iv_df["cnt_0_of_{}".format(value)].sum()
    woe_iv_df['ratio_1_of_{}'.format(value)] = woe_iv_df["cnt_1_of_{}".format(value)]/woe_iv_df["cnt_1_of_{}".format(value)].sum()
    # 计算各个箱的woe值
    woe_iv_df['woe_of_{}'.format(value)] = np.log(woe_iv_df['ratio_0_of_{}'.format(value)]/woe_iv_df['ratio_1_of_{}'.format(value)])
    woe_iv_df['woe_of_{}'.format(value)].replace(-np.inf,-1,inplace=True)
    woe_iv_df['woe_of_{}'.format(value)].replace(np.inf,1,inplace=True)
#     woe_iv_df['iv_of_{}'.format(value)] = (woe_iv_df['ratio_0_of_{}'.format(value)] - woe_iv_df['ratio_1_of_{}'.format(value)]) * woe_iv_df['woe_of_{}'.format(value)]
    woe_iv_df=woe_iv_df[[value,'woe_of_{}'.format(value)]]
    return woe_iv_df

woe_iv_list=["age","using_time"]
for value in tqdm(woe_iv_list):
    data=data.merge(woe_iv_of_value(value),on=value,how="left")

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.31it/s]


In [9]:
# 对base表中的类别特征进行LabelEncoder
for col in tqdm([f for f in data.select_dtypes('object').columns if f not in ['user']]):
    le = LabelEncoder()
    data[col].fillna('-1', inplace=True)
    data[col] = le.fit_transform(data[col])

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 34.69it/s]


In [11]:
# 对base表中的部分字段进行频数编码
DATA_PATH="F:/Desktop/data/"
train_base = pd.read_csv(DATA_PATH+'train_base.csv')
test_a_base = pd.read_csv(DATA_PATH+'test_a_base.csv')
test_b_base=pd.read_csv(DATA_PATH+"testb_base.csv")
df = pd.concat([train_base,test_a_base,test_b_base], axis=0, ignore_index=True)
del train_base, test_a_base, test_b_base
df_category=df.select_dtypes('object')

df_category_nunique = df_category.nunique()
A_cnt_features = [col for col in df_category_nunique.index if df_category_nunique.loc[col] > 5 and col!='user']
frequency_fea = pd.DataFrame()
frequency_fea['user'] = df_category['user'].values
for col in tqdm_notebook(A_cnt_features):
    df_category[col] = df_category[col].fillna(-999)
    frequency_fea[col + '_cnt'] = df_category[col].map(df_category[col].value_counts())
    
data=data.merge(frequency_fea,on="user",how="left")
del df,df_category,df_category_nunique

In [13]:
#定义用户金额特征函数
def gen_user_amount_features(df):
    group_df = df.groupby(['user'])['amount'].agg([
        ('user_amount_mean', 'mean'),
        ('user_amount_std','std'),
        ('user_amount_max', 'max'),
        ('user_amount_min', 'min'),
        ('user_amount_sum', 'sum'),
        ('user_amount_med', 'median'),
        ('user_amount_cnt', 'count')
    ]).reset_index()
    return group_df

#用户金额特征提取
data = data.merge(gen_user_amount_features(trans_df), on=['user'], how='left')

In [14]:
#定义用户字段唯一值函数
def gen_user_nunique_features(df, value, prefix):
    group_df = df.groupby(['user'])[value].agg([
        ('user_{}_{}_nuniq'.format(prefix, value), 'nunique')]
    ).reset_index()
    return group_df

#交易表字段唯一值特征提取
for col in tqdm(['day', 'platform', 'tunnel_in', 'tunnel_out', 'type1', 'type2', 'ip', 'ip_3']):
    data = data.merge(gen_user_nunique_features(df=trans_df, value=col, prefix='trans'), on=['user'], how='left')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.45it/s]


In [15]:
#用户日均交易金额
data['user_amount_per_days'] = data['user_amount_sum'] / data['user_trans_day_nuniq']
#用户平均每次交易金额
data['user_amount_per_cnt'] = data['user_amount_sum'] / data['user_amount_cnt']

In [16]:
#定义用户分组金额交易函数
def gen_user_group_amount_features(df, value):
    group_df = df.pivot_table(index='user',
                              columns=value,
                              values='amount',
                              dropna=False,
                              aggfunc=['count', 'sum'])
    group_df.columns = ['user_{}_{}_amount_{}'.format(value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)

    return group_df

#生成用户关于平台的分组交易金额特征
data = data.merge(gen_user_group_amount_features(df=trans_df, value='platform'), on=['user'], how='left')
#生成用户关于type1的分组交易金额特征
data = data.merge(gen_user_group_amount_features(df=trans_df, value='type1'), on=['user'], how='left')
#生成用户关于type2的分组交易金额特征
data = data.merge(gen_user_group_amount_features(df=trans_df, value='type2'), on=['user'], how='left')
#生成用户关于time的分组交易金额特征
data = data.merge(gen_user_group_amount_features(df=trans_df, value='time'), on=['user'], how='left')
#生成用户关于week的分组交易金额特征
data = data.merge(gen_user_group_amount_features(df=trans_df, value='week'), on=['user'], how='left')

In [17]:
#定义交易天数金额窗口周期函数
def gen_user_window_amount_features(df, window):
    group_df = df[df['day']>window].groupby('user')['amount'].agg([
        ('user_amount_mean_{}d'.format(window), 'mean'),
        ('user_amount_std_{}d'.format(window),'std'),
        ('user_amount_max_{}d'.format(window),'max'),
        ('user_amount_min_{}d'.format(window), 'min'),
        ('user_amount_sum_{}d'.format(window),'sum'),
        ('user_amount_med_{}d'.format(window),'median'),
        ('user_amount_cnt_{}d'.format(window),'count')
    ]).reset_index()
    return group_df

#提取7日窗口金额特征
data = data.merge(gen_user_window_amount_features(df=trans_df, window=7), on=['user'], how='left')
#提取15日窗口金额特征
data = data.merge(gen_user_window_amount_features(df=trans_df, window=15), on=['user'], how='left')
#提取23日窗口金额特征
data = data.merge(gen_user_window_amount_features(df=trans_df, window=23), on=['user'], how='left')

In [18]:
#定义交易小时金额窗口周期函数
def gen_user_window_amount_features(df, window):
    group_df = df[df['hour']>window].groupby('user')['amount'].agg([
        ('user_amount_mean_{}h'.format(window), 'mean'),
        ('user_amount_std_{}h'.format(window),'std'),
        ('user_amount_max_{}h'.format(window),'max'),
        ('user_amount_min_{}h'.format(window),'min'),
        ('user_amount_sum_{}h'.format(window), 'sum'),
        ('user_amount_med_{}h'.format(window), 'median'),
        ('user_amount_cnt_{}h'.format(window), 'count')
    ]).reset_index()
    return group_df

#提取6时窗口金额特征
data = data.merge(gen_user_window_amount_features(df=trans_df, window=6), on=['user'], how='left')
#提取12时窗口金额特征
data = data.merge(gen_user_window_amount_features(df=trans_df, window=12), on=['user'], how='left')
#提取18时窗口金额特征
data = data.merge(gen_user_window_amount_features(df=trans_df, window=18), on=['user'], how='left')

In [19]:
#定义空值特征提取函数
def gen_user_null_features(df, value, prefix):
    df['is_null'] = 0
    df.loc[df[value].isnull(), 'is_null'] = 1

    group_df = df.groupby(['user'])['is_null'].agg([('user_{}_{}_null_cnt'.format(prefix, value), 'sum'),
                                                    ('user_{}_{}_null_ratio'.format(prefix, value),'mean')]).reset_index()
    return group_df

#提取交易表ip地址缺失特征
data = data.merge(gen_user_null_features(df=trans_df, value='ip', prefix='trans'), on=['user'], how='left')

In [20]:
#提取type1为45a1168437c708ff的用户交易最小天数特征
group_df = trans_df[trans_df['type1']=='45a1168437c708ff'].groupby(['user'])['day'].agg([('user_type1_45a1168437c708ff_min_day', 'min')]).reset_index()
data = data.merge(group_df, on=['user'], how='left')
del group_df

In [21]:
#定义用户每天每小时交易金额函数
def per_hour_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('per_hour_amt_sum','sum'),
        ('per_hour_amt_mean','mean'),
        ('per_hour_amt_max','max'),
        ('per_hour_amt_min','min'),
        ('per_hour_amt_cnt','count'),
        ('per_hour_amt_std','std')
    ]).reset_index()
    return group_df

#生成用户每天每小时交易金额特征
data = data.merge(per_hour_amt(trans_df,value1="day",value2="hour"), on=['user'], how='left')

In [22]:
#定义用户来源与去向交易金额函数
def tunnel_in_out_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('tunnel_in_out_amt_sum','sum'),
        ('tunnel_in_out_amt_cnt','count'),
        ('tunnel_in_out_amt_mean','mean'),
        ('tunnel_in_out_amt_max','max'),
        ('tunnel_in_out_amt_min','min'),
        ('tunnel_in_out_amt_std','std'),
    ]).reset_index()
    return group_df

#生成用户来源去向交易金额特征
data = data.merge(tunnel_in_out_amt(trans_df,value1="tunnel_in",value2="tunnel_out"), on=['user'], how='left')

In [23]:
#定义用户日期ip金额函数
def day_ip_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('day_ip_amt_sum','sum'),
        ('day_ip_amt_mean','mean'),
        ('day_ip_amt_std','std'),
        ('day_ip_amt_max','max'),
        ('day_ip_amt_min','min'),
        ('day_ip_amt_cnt','count')
    ]).reset_index()
    return group_df

#生成用户日期ip交易金额特征
data = data.merge(day_ip_amt(trans_df,value1="day",value2="ip"), on=['user'], how='left')

In [24]:
#定义用户前后时间交易金额gap函数
def amt_gap(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['amount_gap']=abs(group_df["amount"]-group_df["last_amount"])
    group_df=group_df[['user','amount_gap']]
    group_df=group_df.groupby('user')['amount_gap'].agg([
        ('amt_gap_sum','sum'),
        ('amt_gap_mean','mean'),
        ('amt_gap_std','std'),
        ('amt_gap_max','max'),
        ('amt_gap_min','min')
    ]).reset_index()
    return group_df

#生成用户日期gap交易金额特征
data = data.merge(amt_gap(trans_df,value1="day",value2="hour"), on=['user'], how='left')

In [25]:
#定义交易金额gap比率和函数
def gap_amt_rate(df,value1,value2):
    group_df=df[['user',value1,value2,'amount']]
    group_df=group_df.groupby(['user', 'day', 'hour'])['amount'].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['gap_rate']=group_df['amount']/group_df['last_amount']
    group_df=group_df[['user','gap_rate']]
    group_df=group_df.groupby('user')['gap_rate'].agg('sum').reset_index()
    return group_df

#生成用户日期交易gap比率和特征
data = data.merge(gap_amt_rate(trans_df,value1="day",value2="hour"), on=['user'], how='left')

In [26]:
#定义用户每天交易金额函数
def per_day_amt(df,value):
    group_df=df[['user',value, 'amount']]
    group_df=group_df.groupby(['user',value])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('per_day_amt_mean','mean'),
        ('per_day_amt_max','max'),
        ('per_day_amt_min','min'),
        ('per_day_amt_std','std')
    ]).reset_index()
    return group_df

#生成用户每天交易金额特征
data = data.merge(per_day_amt(trans_df,value="day"), on=['user'], how='left')

In [27]:
#定义用户前后时间交易金额gap函数
def day_amt_gap(df,value):
    group_df=df[['user',value, 'amount']]
    group_df=group_df.groupby(['user',value])["amount"].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['amount_gap']=abs(group_df["amount"]-group_df["last_amount"])
    group_df=group_df[['user','amount_gap']]
    group_df=group_df.groupby('user')['amount_gap'].agg([
        ('day_amt_gap_mean','mean'),
        ('day_amt_gap_std','std'),
        ('day_amt_gap_max','max'),
        ('day_amt_gap_min','min')
    ]).reset_index()
    return group_df

#生成用户每天交易金额gap特征
data = data.merge(day_amt_gap(trans_df,value="day"), on=['user'], how='left')

In [28]:
#定义每天交易金额gap比率和函数
def day_gap_amt_rate(df,value):
    group_df=df[['user',value,'amount']]
    group_df=group_df.groupby(['user', value])['amount'].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['day_gap_rate']=group_df['amount']/group_df['last_amount']
    group_df=group_df[['user','day_gap_rate']]
    group_df=group_df.groupby('user')['day_gap_rate'].agg('sum').reset_index()
    return group_df

#生成用户每天交易金额gap比率和特征
data = data.merge(day_gap_amt_rate(trans_df,value="day"), on=['user'], how='left')

In [29]:
#定义用户日期type1金额函数
def day_type_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('day_type1_amt_sum','sum'),
        ('day_type1_amt_mean','mean'),
        ('day_type1_amt_std','std'),
        ('day_type1_amt_max','max'),
        ('day_type1_amt_min','min'),
        ('day_type1_amt_cnt','count')
    ]).reset_index()
    return group_df

#生成用户每天type1交易金额特征
data = data.merge(day_type_amt(trans_df,value1="day",value2="type1"), on=['user'], how='left')

In [30]:
#定义用户日期type2金额函数
def day_type2_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('day_type2_amt_sum','sum'),
        ('day_type2_amt_mean','mean'),
        ('day_type2_amt_std','std'),
        ('day_type2_amt_max','max'),
        ('day_type2_amt_min','min'),
        ('day_type2_amt_cnt','count')
    ]).reset_index()
    return group_df

#生成用户每天type交易金额特征
data = data.merge(day_type2_amt(trans_df,value1="day",value2="type2"), on=['user'], how='left')

In [32]:
from gensim.models import Word2Vec
import multiprocessing
def w2v_feat(data_frame, feat, mode):
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    print(f'Start {mode} word2vec ...')
    model = Word2Vec(data_frame[feat].values.tolist(), size=5, window=2, min_count=1,
                     workers=multiprocessing.cpu_count(), iter=10)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'w2c_trans_{t}_{n}' for n in range(5)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all

#生成word2vec特征
trans_feat=["platform","tunnel_in","tunnel_out","amount","type1","type2","ip","day","hour"]
data=data.merge(w2v_feat(trans_df,trans_feat,'trans'), on=['user'], how='left')

Start trans word2vec ...
Start gen feat of platform ...
Start gen feat of tunnel_in ...
Start gen feat of tunnel_out ...
Start gen feat of amount ...
Start gen feat of type1 ...
Start gen feat of type2 ...
Start gen feat of ip ...
Start gen feat of day ...
Start gen feat of hour ...


In [33]:
#对交易表部分字段进行CountEncoder
from category_encoders.count import CountEncoder
for i in ["ip","ip_3","amount"]:
    trans_df["count_{}_trans".format(i)]=CountEncoder().fit_transform(trans_df[i])
    group_df=trans_df.groupby('user')["count_{}_trans".format(i)].agg([
        ("count_{}_trans_max".format(i),'max'),
        ("count_{}_trans_min".format(i), 'min'),
        ("count_{}_trans_mean".format(i),'mean'),
        ("count_{}_trans_std".format(i),'std')
    ]).reset_index()
    data=data.merge(group_df,on="user",how="left")
    
trans_df.drop(columns=trans_df.columns[-3:],inplace=True)

In [35]:
#交易信息表部分字段频数编码
trans_df_category=trans_df.drop(columns=["tm_diff","time"])
trans_df_category_nunique = trans_df_category.nunique()
A_cnt_features = [col for col in trans_df_category_nunique.index if trans_df_category_nunique.loc[col] > 5 and col!='user']
len(A_cnt_features)
frequency_fea = pd.DataFrame()
frequency_fea['user'] = trans_df_category['user'].values
for col in tqdm_notebook(A_cnt_features):
    trans_df_category[col] = trans_df_category[col].fillna(-999)
    frequency_fea[col + '_cnt'] = trans_df_category[col].map(trans_df_category[col].value_counts())
    
for i in tqdm_notebook(frequency_fea.columns[1:]):
    group_df=frequency_fea[["user",i]]
    group_df=group_df.groupby("user")[i].agg([
        ('freq_{}_max'.format(i),'max'),
        ('freq_{}_min'.format(i),'min'),
        ('freq_{}_mean'.format(i),'mean'),
        ('freq_{}_std'.format(i),'std')]).reset_index()
    data=data.merge(group_df,on="user",how="left")

In [36]:
def gen_user_tfidf_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_tfidf_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

In [37]:
def gen_user_countvec_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = CountVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_countvec_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

In [38]:
data = data.merge(gen_user_tfidf_features(df=op_df, value='op_mode'), on=['user'], how='left')
data = data.merge(gen_user_tfidf_features(df=op_df, value='op_type'), on=['user'], how='left')
data = data.merge(gen_user_countvec_features(df=op_df, value='op_mode'), on=['user'], how='left')
data = data.merge(gen_user_countvec_features(df=op_df, value='op_type'), on=['user'], how='left')

In [39]:
from gensim.models import Word2Vec
import multiprocessing
def w2v_feat(data_frame, feat, mode):
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    print(f'Start {mode} word2vec ...')
    model = Word2Vec(data_frame[feat].values.tolist(), size=5, window=2, min_count=1,
                     workers=multiprocessing.cpu_count(), iter=10)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'w2c_op_{t}_{n}' for n in range(5)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all

#生成word2vec特征
op_feat=["op_type","op_mode","op_device","ip","channel","day","hour"]
data=data.merge(w2v_feat(op_df,op_feat,'op'), on=['user'], how='left')

Start op word2vec ...
Start gen feat of op_type ...
Start gen feat of op_mode ...
Start gen feat of op_device ...
Start gen feat of ip ...
Start gen feat of channel ...
Start gen feat of day ...
Start gen feat of hour ...


In [40]:
#对操作表部分字段进行CountEncoder
from category_encoders.count import CountEncoder
for i in ["ip","ip_3","op_device","op_type","op_mode"]:
    op_df["count_{}_op".format(i)]=CountEncoder().fit_transform(op_df[i])
    group_df=op_df.groupby('user')["count_{}_op".format(i)].agg([
        ("count_{}_op_max".format(i),'max'),
        ("count_{}_op_min".format(i), 'min'),
        ("count_{}_op_mean".format(i),'mean'),
        ("count_{}_op_std".format(i),'std')
    ]).reset_index()
    data=data.merge(group_df,on="user",how="left")
    
op_df.drop(columns=op_df.columns[-5:],inplace=True)

In [41]:
#操作表频数编码
op_df_category=op_df.drop(columns=["tm_diff","time"])
op_df_category_nunique = op_df_category.nunique()
A_cnt_features = [col for col in op_df_category_nunique.index if op_df_category_nunique.loc[col] > 5 and col!='user']
len(A_cnt_features)
frequency_fea = pd.DataFrame()
frequency_fea['user'] = op_df_category['user'].values
for col in tqdm_notebook(A_cnt_features):
    op_df_category[col] = op_df_category[col].fillna(-999)
    frequency_fea[col + '_cnt'] = op_df_category[col].map(op_df_category[col].value_counts())

for i in tqdm_notebook(frequency_fea.columns[1:]):
    group_df=frequency_fea[["user",i]]
    group_df=group_df.groupby("user")[i].agg([
       ('freq_{}_max'.format(i),'max'),
        ('freq_{}_min'.format(i),'min'),
        ('freq_{}_mean'.format(i),'mean'),
        ('freq_{}_std'.format(i),'std')]).reset_index()
    data=data.merge(group_df,on="user",how="left")

In [43]:
#定义用户字段唯一值函数
def gen_user_nunique_features(df, value, prefix):
    group_df = df.groupby(['user'])[value].agg([
        ('user_{}_{}_nuniq'.format(prefix, value),'nunique')
    ]).reset_index()
    return group_df

#操作表字段唯一值特征提取
for col in tqdm(['op_type', 'op_mode', 'ip', 'channel', 'ip_3', 'day']):
    data = data.merge(gen_user_nunique_features(df=op_df, value=col, prefix='op'), on=['user'], how='left')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:12<00:00,  2.06s/it]


In [44]:
#定义天数操作窗口周期函数
def gen_user_window_op_features(df, window):
    group_df = df[df['day']>window].groupby('user')['op_type'].agg([
        ('user_op_cnt_{}d'.format(window),'count')
    ]).reset_index()
    return group_df

op_df["day"]=op_df["day"].astype(int)
#提取5日窗口操作数特征
data = data.merge(gen_user_window_op_features(df=op_df, window=5), on=['user'], how='left')
#提取10日窗口操作数特征
data = data.merge(gen_user_window_op_features(df=op_df, window=10), on=['user'], how='left')

In [45]:
#定义小时op窗口周期函数
def gen_op_window_hour_features(df, window):
    group_df = df[df['hour']>window].groupby('user')['op_type'].agg([
        ('user_op_cnt_{}h'.format(window),'count')]).reset_index()
    return group_df

op_df["hour"]=op_df["hour"].astype(int)
#提取6时窗口op特征
data = data.merge(gen_op_window_hour_features(df=op_df, window=6), on=['user'], how='left')
#提取12时窗口op特征
data = data.merge(gen_op_window_hour_features(df=op_df, window=12), on=['user'], how='left')
#提取18时窗口op特征
data = data.merge(gen_op_window_hour_features(df=op_df, window=18), on=['user'], how='left')

In [46]:
#定义用户分组op函数
def gen_user_group_op_features(df, value):
    group_df = df.pivot_table(index='user',
                              columns=value,
                              values='op_type',
                              dropna=False,
                              aggfunc=['count'])
    group_df.columns = ['user_{}_{}_op_{}'.format(value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)

    return group_df

#生成用户关于time的分组op特征
data = data.merge(gen_user_group_op_features(df=op_df, value='time'), on=['user'], how='left')

In [47]:
#每一个用户操作次数
op_count = op_df[['user']]
op_count['op_count'] = 1
op_count = op_count.groupby('user').agg('count').reset_index()
data = pd.merge(data, op_count, on='user', how='left')
del op_count

In [48]:
data["op_cnt_per_day"]=data["op_count"]/data["user_op_day_nuniq"]

In [49]:
# 每一个用户每一天每一小时操作数的最大值，最小值，均值，标准差
def day_per_hour_cnt(df,value1,value2):
    group_df = op_df[['user', 'day', 'hour']]
    group_df['everyday_everyhour'] = 1
    group_df = group_df.groupby(['user', 'day', 'hour']).agg('count').reset_index()
    group_df = group_df.drop(['day', 'hour'],axis = 1)
    group_df = group_df.groupby('user')['everyday_everyhour'].agg([
        ('day_per_hour_mean','mean'),
        ('day_per_hour_max','max'),
        ('day_per_hour_min','min'), 
        ('day_per_hour_std','std')]).reset_index()
    return group_df

data = data.merge(day_per_hour_cnt(op_df,"day","hour"), on='user', how='left')

In [50]:
# 每一个用户每一天操作数的最大值，最小值，均值，标准差
frequence_one_day = op_df[['user', 'day']]
frequence_one_day['everyday'] = 1
frequence_one_day = frequence_one_day.groupby(['user', 'day']).agg('count').reset_index()
frequence_one_day = frequence_one_day.drop('day', axis=1)
frequence_one_day = frequence_one_day.groupby('user')['everyday'].agg([
    ('per_day_mean','mean'),
    ('per_day_max','max'),
    ('per_day_min','min'),
    ('per_day_std','std')]).reset_index()
data = data.merge(frequence_one_day, on='user', how='left')

In [51]:
#morning op count
frequence_morning = op_df[op_df.time=="morning"][['user', 'day','hour']]
frequence_morning['everyday_morning'] = 1
frequence_morning = frequence_morning.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_morning = frequence_morning.groupby(['user', 'day'])['everyday_morning'].agg('sum').reset_index()
frequence_morning = frequence_morning[['user', 'everyday_morning']]
frequence_morning = frequence_morning.groupby('user')['everyday_morning'].agg([
    ('per_mor_mean','mean'),
    ('per_mor_max','max'),
    ('per_mor_min','min'),
    ('per_mor_std','std')]).reset_index()
data = data.merge(frequence_morning, on='user', how='left')
del frequence_morning

In [52]:
#afternoon op count
frequence_afternoon = op_df[op_df.time=="afternoon"][['user', 'day','hour']]
frequence_afternoon['everyday_afternoon'] = 1
frequence_afternoon = frequence_afternoon.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_afternoon = frequence_afternoon.groupby(['user', 'day'])['everyday_afternoon'].agg('sum').reset_index()
frequence_afternoon = frequence_afternoon[['user', 'everyday_afternoon']]
frequence_afternoon = frequence_afternoon.groupby('user')['everyday_afternoon'].agg([
    ('per_after_mean','mean'),
    ('per_after_max','max'),
    ('per_after_min','min'),
    ('per_after_std','std')]).reset_index()
data = data.merge(frequence_afternoon, on='user', how='left')
del frequence_afternoon

In [53]:
#evening op count
frequence_evening = op_df[op_df.time=="evening"][['user', 'day','hour']]
frequence_evening['everyday_evening'] = 1
frequence_evening = frequence_evening.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_evening = frequence_evening.groupby(['user', 'day'])['everyday_evening'].agg('sum').reset_index()
frequence_evening = frequence_evening[['user', 'everyday_evening']]
frequence_evening = frequence_evening.groupby('user')['everyday_evening'].agg([
    ('per_eve_mean','mean'),
    ('per_eve_max','max'),
    ('per_eve_min','min'),
    ('per_eve_std','std')]).reset_index()
data = data.merge(frequence_evening, on='user', how='left')
del frequence_evening

In [54]:
#night op count
frequence_night = op_df[op_df.time=="night"][['user', 'day','hour']]
frequence_night['everyday_night'] = 1
frequence_night = frequence_night.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_night = frequence_night.groupby(['user', 'day'])['everyday_night'].agg('sum').reset_index()
frequence_night = frequence_night[['user', 'everyday_night']]
frequence_night = frequence_night.groupby('user')['everyday_night'].agg([
    ('per_night_mean','mean'),
    ('per_night_max','max'),
    ('per_night_min','min'),
    ('per_night_std','std')]).reset_index()
data = data.merge(frequence_night, on='user', how='left')
del frequence_night

In [55]:
# 每一个用户这一天距离前面一段时间操作数的最大值，最小值，均值，标准差
frequence_one_day_gap = op_df[['user', 'day']]
frequence_one_day_gap['everyday'] = 1
frequence_one_day_gap = frequence_one_day_gap.groupby(['user', 'day']).agg('count').reset_index()
frequence_one_day_gap['everyday_before'] = frequence_one_day_gap.groupby('user')['everyday'].shift(1)
frequence_one_day_gap['everyday_before_gap'] = abs(frequence_one_day_gap['everyday'] - frequence_one_day_gap['everyday_before'])
frequence_one_day_gap = frequence_one_day_gap[['user', 'everyday_before_gap']].groupby('user')['everyday_before_gap'].agg([
    ('op_day_gap_mean','mean'),
    ('op_day_gap_max','max'),
    ('op_day_gap_min','min'),
    ('op_day_gap_std','std')]).reset_index()
data = data.merge(frequence_one_day_gap, on='user', how='left')
del frequence_one_day_gap

In [56]:
# 每一个用户这一小时距离前面一段时间操作数的最大值，最小值，均值，标准差
frequence_one_hour_gap = op_df[['user', 'day', 'hour']]
frequence_one_hour_gap['everyhour'] = 1
frequence_one_hour_gap = frequence_one_hour_gap.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_one_hour_gap['everyhour_before'] = frequence_one_hour_gap.groupby('user')['everyhour'].shift(1)
frequence_one_hour_gap['everyhour_before_gap'] = abs(frequence_one_hour_gap['everyhour'] - frequence_one_hour_gap['everyhour_before'])
frequence_one_hour_gap = frequence_one_hour_gap[['user', 'everyhour_before_gap']].groupby('user')['everyhour_before_gap'].agg([
    ('hour_gap_mean','mean'),
    ('hour_gap_max','max'),
    ('hour_gap_min','min'),
    ('hour_gap_std','std')]).reset_index()
data = data.merge(frequence_one_hour_gap, on='user', how='left')
del frequence_one_hour_gap

In [57]:
# 每一个用户这一天距离前面一段时间操作数的比率和
frequence_one_day_rate = op_df[['user', 'day']]
frequence_one_day_rate['everyday'] = 1
frequence_one_day_rate = frequence_one_day_rate.groupby(['user', 'day']).agg('count').reset_index()
frequence_one_day_rate['everyday_before'] = frequence_one_day_rate.groupby('user')['everyday'].shift(1)
frequence_one_day_rate['everyday_before_rate'] = frequence_one_day_rate['everyday'] / frequence_one_day_rate['everyday_before']
frequence_one_day_rate = frequence_one_day_rate[['user', 'everyday_before_rate']].groupby('user')['everyday_before_rate'].agg('sum').reset_index()
data = data.merge(frequence_one_day_rate, on='user', how='left')
del frequence_one_day_rate

In [58]:
# 每一个用户这一小时距离前面一段时间操作数的比率和
frequence_one_hour_rate = op_df[['user', 'day', 'hour']]
frequence_one_hour_rate['everyhour'] = 1
frequence_one_hour_rate = frequence_one_hour_rate.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_one_hour_rate['everyhour_before'] = frequence_one_hour_rate.groupby('user')['everyhour'].shift(1)
frequence_one_hour_rate['everyhour_before_rate'] = frequence_one_hour_rate['everyhour'] - frequence_one_hour_rate['everyhour_before']
frequence_one_hour_rate = frequence_one_hour_rate[['user', 'everyhour_before_rate']].groupby('user')['everyhour_before_rate'].agg('sum').reset_index()
data = data.merge(frequence_one_hour_rate, on='user', how='left')
del frequence_one_hour_rate

In [59]:
# 每秒用户的操作最大值、最小值、均值、方差
every_second = op_df[['user', 'second']]
every_second['operation_second_caozuo'] = 1
every_second = every_second.groupby(['user', 'second']).agg('count').reset_index()
every_second = every_second[['user', 'operation_second_caozuo']]
every_second = every_second.groupby('user')['operation_second_caozuo'].agg([
    ('per_sec_max','max'),
    ('per_sec_min','min'),
    ('per_sec_mean','mean'),
    ('per_sec_std','std')]).reset_index()
data = data.merge(every_second, on='user', how='left')
del every_second

In [60]:
# 每分钟用户操作的最大值、最小值、均值、方差
every_minute = op_df[['user', 'min']]
every_minute['operation_minute_caozuo'] = 1
every_minute = every_minute.groupby(['user', 'min']).agg('count').reset_index()
every_minute = every_minute[['user', 'operation_minute_caozuo']]
every_minute = every_minute.groupby('user')['operation_minute_caozuo'].agg([
    ('per_minute_max','max'),
    ('per_minute_min','min'),
    ('per_minute_mean','mean'),
    ('per_minute_std','std')]).reset_index()
data = data.merge(every_minute, on='user', how='left')
del every_minute

In [61]:
#per day device nunique
dev_per_day_cnt=op_df[op_df.op_device!="nan"][["user","day","op_device"]].drop_duplicates()
dev_per_day_cnt=pd.DataFrame(dev_per_day_cnt.groupby(["user","day"])["op_device"].nunique()).reset_index()
dev_per_day_cnt=dev_per_day_cnt[["user","op_device"]]
dev_per_day_cnt=dev_per_day_cnt.groupby(['user'])["op_device"].agg([
    ("dev_nun_mean","mean"),
    ("dev_nun_max","max"),
    ("dev_nun_min","min"),
    ("dev_nun_std","std")
]).reset_index()
data=data.merge(dev_per_day_cnt, on='user', how='left')
del dev_per_day_cnt

In [62]:
#per day hour device nunique
dev_per_hour_cnt=op_df[op_df.op_device!="nan"][["user","day","hour","op_device"]].drop_duplicates()
dev_per_hour_cnt=pd.DataFrame(dev_per_hour_cnt.groupby(["user","day","hour"])["op_device"].nunique()).reset_index()
dev_per_hour_cnt=dev_per_hour_cnt[["user","op_device"]]
dev_per_hour_cnt=dev_per_hour_cnt.groupby(['user'])["op_device"].agg([
    ("dev_hour_nun_mean","mean"),
    ("dev_hour_nun_max","max"),
    ("dev_hour_nun_min","min"),
    ("dev_hour_nun_std","std")
]).reset_index()
data=data.merge(dev_per_hour_cnt, on='user', how='left')
del dev_per_hour_cnt

In [63]:
#per day ip nunique
ip_per_day_cnt=op_df[op_df.ip!="nan"][["user","day","ip"]].drop_duplicates()
ip_per_day_cnt=pd.DataFrame(ip_per_day_cnt.groupby(["user","day"])["ip"].nunique()).reset_index()
ip_per_day_cnt=ip_per_day_cnt[["user","ip"]]
ip_per_day_cnt=ip_per_day_cnt.groupby(['user'])["ip"].agg([
    ("ip_nun_mean","mean"),
    ("ip_nun_max","max"),
    ("ip_nun_mean","mean"),
    ("ip_nun_std","std")
]).reset_index()
data=data.merge(ip_per_day_cnt, on='user', how='left')
del ip_per_day_cnt

In [64]:
#per day hour ip nunique
ip_per_hour_cnt=op_df[op_df.ip!="nan"][["user","day","hour","ip"]].drop_duplicates()
ip_per_hour_cnt=pd.DataFrame(ip_per_hour_cnt.groupby(["user","day","hour"])["ip"].nunique()).reset_index()
ip_per_hour_cnt=ip_per_hour_cnt[["user","ip"]]
ip_per_hour_cnt=ip_per_hour_cnt.groupby(['user'])["ip"].agg([
    ("ip_hour_nun_max","max"),
    ("ip_hour_nun_min","min"),
    ("ip_hour_nun_mean","mean"),
    ("ip_hour_nun_std","std")
]).reset_index()
data=data.merge(ip_per_hour_cnt, on='user', how='left')
del ip_per_hour_cnt
# ip_per_hour_cnt

In [65]:
#per day hour min ip nunique
ip_per_min_cnt=op_df[op_df.ip!="nan"][["user","day","hour","min","ip"]].drop_duplicates()
ip_per_min_cnt=pd.DataFrame(ip_per_min_cnt.groupby(["user","day","hour","min"])["ip"].nunique()).reset_index()
ip_per_min_cnt=ip_per_min_cnt[["user","ip"]]
ip_per_min_cnt=ip_per_min_cnt.groupby(['user'])["ip"].agg([
    ("ip_min_nun_max","max"),
    ("ip_min_nun_min","min"),
    ("ip_min_nun_mean","mean"),
    ("ip_min_nun_std","std")
]).reset_index()
data=data.merge(ip_per_min_cnt, on='user', how='left')
del ip_per_min_cnt
# ip_per_min_cnt

In [119]:
#保存特征文件
data.to_csv("F:/data.csv",index=False)

In [120]:
data.shape

(96202, 830)